In [1]:
import os
import subprocess

import xarray as xr  
import numpy as np
import pandas as pd
import pickle

import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import regularizers
from tensorflow.keras import metrics
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
import tensorflow.keras.backend as kb

import keras

import matplotlib.pyplot as plt
from global_land_mask import globe

from sklearn import linear_model
import scipy.stats as ss

import random
import seaborn as sb
sb.set_style("white")

Using TensorFlow backend.


In [2]:
# Makes these global for potential editing within scripts
BASEVAR = sorted(['tas', 'pr']) # the variables used for the ANN, always alphabetical 
PRIM = 'cmip6' #primary dataset for model training and validation
#observational dataset for model testing, dictionary from each VAR
OBS = {'tas':'BEST','pr':'JRA55', 'cdd':'BEST', 'hdd':'BEST',
       'r95t':'ERA5', 'r95p':'ERA5', 'r99p':'ERA5', 'r99t':'ERA5'
      } 

TIMETYPE = 'annual' #are we using just 'annual' mean data? Means each 'season'? Means each 'month'?
TIMERANGE = ['1920','2099']
BASELINETIMERANGE = [1980, 2019]
RES = 'r90x45' # the resolution of the data, format: r90x45
REGION = 'globe-land' #'NA-land' # 'north_atl' # Define region in get_region_bounds, None is global

if TIMETYPE == 'month':
    varlist = []
    for var in BASEVAR:
        varlist.extend( [var + '-%02i' % num for num in np.arange(12)+1] )
    VAR = sorted(varlist)
else:
    VAR = BASEVAR

# Computer username
USER = 'Jamin' #laptop
#USER = 'jrader' #work

ROOT = '/Users/' + USER + '/Rams/TOE_Research/' # root directory for the whole project
PATH = ROOT + 'data/' # the path of the netCDF files (before PRIM)
STAND_TYPE = 'classification/'
NETCDF_PATH = PATH + 'netcdfFiles_' + RES + '/'
NPZ_PATH = PATH + 'npzFiles_' + RES + '/' # Don't change this format
MOD_DIR = 'models/' + STAND_TYPE
PLOT_DIR = 'plots/' + STAND_TYPE
SAVE_LRP_PATH = PATH + 'lrp_npz_' + RES + '/' + STAND_TYPE
MASK = PATH + 'landmask/landmask_nopole_' + RES + '.npz'

MODEL_NUM = None # only used if loading a previously saved model
NEW_MODEL_NUM = None # used to overwrite a model number if model doesn't already exist

VAL_SPLIT = .2 # what fraction of the models should be used for validation?
CLOSS_YEAR = np.nan # setting the years for the loss function equal to nan for error checking
VAL_MODELS = None #[0,3, 9, 11] #[0,3, 9, 11] < for pr #[0,2, 6, 13, 15, 22] < for tas
    
REMOVE_GLOBAL_MEAN = False # Cannot do -land if REMOVE_GLOBAL_MEAN is true

ACTIVATION = 'relu' # The activation function used for the ANN
RIDGE_PENALTY = .1 #10 (for NA-land)

#['tas'] 
VAL_MODELS = sorted([ 9,7, 29, 11, 34, 15, 31])
RIDGE_PENALTY = .01
#['pr', 'tas'] [ 9  5 22  3 17 30 21]
#['pr'] [30 24 17 12 28  2 20]
#['tas-01', 'tas-02', 'tas-03', 'tas-04', 'tas-05', 'tas-06', 'tas-07', 'tas-08', 'tas-09', 'tas-10', 'tas-11', 'tas-12'] [16 13 11 34 29 10 27]
#['pr-01', 'pr-02', 'pr-03', 'pr-04', 'pr-05', 'pr-06', 'pr-07', 'pr-08', 'pr-09', 'pr-10', 'pr-11', 'pr-12', 'tas-01', 'tas-02', 'tas-03', 'tas-04', 'tas-05', 'tas-06', 'tas-07', 'tas-08', 'tas-09', 'tas-10', 'tas-11', 'tas-12'] [16 13 11 34 29 10 27]
#['pr-01', 'pr-02', 'pr-03', 'pr-04', 'pr-05', 'pr-06', 'pr-07', 'pr-08', 'pr-09', 'pr-10', 'pr-11', 'pr-12'] [29  1 15  0 11  2 14]
#['tas-djf', 'tas-jja', 'tas-mam', 'tas-son'] [31 17 27 12 22 29 13]
#['pr-djf', 'pr-jja', 'pr-mam', 'pr-son'] [ 6 20 13 10 31 17 19]
#['pr-djf', 'pr-jja', 'pr-mam', 'pr-son', 'tas-djf', 'tas-jja', 'tas-mam', 'tas-son'] [12 26  1 15 31  7  3]
    
    
LASSO = 0
LEARNING_RATE = 0.001 #.001
EPOCHS = 150
LAYERS = [10,10] #[10,10] has two layers with 10 nodes each, [] is linear
RANDOM_NETWORK_SEED = 41829

# Temporary Values

# random numbers
%env PYTHONHASHSEED=99
np.random.seed(seed=None)
tf.random.set_seed(99)


env: PYTHONHASHSEED=99


# Functions

In [3]:
####### TOE_functions, Jamin Rader ########

#####
### These functions are all for preparing the data (see prepare_data() below)
### Specifically, it takes the netcdf files at PATH, standardizes the data, 
### removes the annual mean, flattens it
#####
 
# User-defined regions
region_bounds = {
                'contig_us': {'lat':(25,50), 'lon':(235,295)},
                'north_atl': {'lat':(20,65), 'lon':(290, 350)},
                'NA': {'lat':(13,75), 'lon':(190, 304)},
                'globe': {'lat':(-90,90), 'lon':(0, 360)},
                'west' : {'lat':(-90,90), 'lon':(190, 352)},
                'americas' : {'lat':(-55,90), 'lon':(190, 335)}
                }
# Adds "land-only" option to region bounds dictionary
reg_keys = [i for i in region_bounds.keys()]
for i in reg_keys:
    region_bounds[i + '-land'] = region_bounds[i].copy()
                    
# Extracts the netcdf data, returns a list of all datasets
def get_nc_data():
    filenames = []
    datasets = []
    # List of lists with all filenames for each variable in each list with variable off the front
    for var in BASEVAR:
        fnames = []
        for fn in os.listdir(NETCDF_PATH+PRIM + '/' + var):
            if fn.endswith('.nc'):
                fnames.append(fn[len(var):])
        filenames.append(fnames)
        
    # Removes dislike filenames
    alike  = set(filenames[0])
    for i in range(1, len(filenames)):
        alike = alike & set(filenames[i])
    # 1-D list with all similar filenames, with variable off front
    filenames = sorted(list(alike))
    # Opening the netcdfs - each list element of 'datasets' is a netcdf file with all variables for given model
    for i in range(len(filenames)):
        fns = [NETCDF_PATH + PRIM + '/' + var + '/' + var + filenames[i] for var in BASEVAR]
        data = xr.open_mfdataset(fns, combine='by_coords', compat='override')
        #try:
        #    data = xr.open_mfdataset(fns, combine='by_coords', compat='override')
        #except:
        #    data = xr.open_mfdataset(fns, combine='nested', concat_dim='time')
        datasets.append(data)
    return datasets

def select_region(xr_array):

    if REGION == None:
        return xr_array
    lat_bounds = region_bounds[REGION]['lat']
    lon_bounds = region_bounds[REGION]['lon']
    xr_return = xr_array.sel(lat=slice(lat_bounds[0], lat_bounds[1])).sel(lon=slice(lon_bounds[0], lon_bounds[1]))
    return xr_return # CHANGED
    #if str(REGION).endswith('-land'):
        #maskinfo = np.load(MASK)
        #boolland, boollats, boollons = maskinfo['mask'], maskinfo['lats'], maskinfo['lons']
        #print('NEXT')
        #print(xr_return.values)
        #print(boolland)
        #boolland = np.stack(np.shape(xr_return)[0]*[boolland], axis=0)
        #xr_return = xr_return.copy() * (boolland * 1) # sets non land regions to zero
        #xr_return = xr_return.where(boolland == False, np.nan)
        
    return xr_return

def time_mean(xr_array):
    # Takes monthly averages and turns them into seasonal, annual averages
    if TIMETYPE == 'annual':
        array = xr_array.groupby("time.year").mean("time").values # Takes annual mean
    elif TIMETYPE == 'month':
        array = xr_array.values
    else: 
        print('timetype is undefined')# and the month thing here later
    return array

def mean_and_flatten(datasets):
    # Loop through datasets and turn month means into annual means
    data, years, models, lats, lons = [], [], [], [],[]
    for ds in datasets:
        flat_arrays = []
        try:
            models.append(ds.model_id)
        except:
            models.append(ds.source_id)
        #print(ds.source_id)
        ds = ds.sel(time=slice(TIMERANGE[0], TIMERANGE[1])) # Selects only the years specified by TIMERANGE

        for var in BASEVAR:
            #var = var.split('-')[0] #if separated by season or something
            xr_array = select_region(ds[var])
            if REMOVE_GLOBAL_MEAN:
                xr_array = xr_array - xr_array.mean(['lat', 'lon'], skipna=True)
            array = time_mean(xr_array)
            if TIMETYPE == 'month':
                flat_array = np.reshape(array, (np.shape(array)[0]//12, -1 )) #flatten if months
            else:
                flat_array = np.reshape(array, (np.shape(array)[0], -1 )) #flatten
            flat_arrays.append(flat_array) # appends flattened variable array
            if var == BASEVAR[-1]:
                lats = xr_array.lat.values
                lons = xr_array.lon.values
        flat_data = np.concatenate(flat_arrays, axis=1) # concatenates all the arrays together
        data.append(flat_data)
        years.append(sorted(list(set(ds.time.dt.year.values)))) # adds model years as list index
        
    return data, years, models, lats, lons

def prepare_data():
    datasets = get_nc_data()
    data, years, models, lats, lons = mean_and_flatten(datasets)
    return data, years, models, lats, lons

# Getting numpy arrays of the observational data

def get_obs():
    
    try:

        datasets = []

        # List of lists with all filenames for each variable in each list with variable off the front
        filenames = [NETCDF_PATH + 'observations/' + OBS[var.split('-')[0]] + '/' + var + '/' 
                     + var + '.' + OBS[var.split('-')[0]].lower() + '.' + 'monmean_' + RES + '.nc' 
                     for var in BASEVAR]

        # Opening the netcdfs - each list element of 'datasets' is a netcdf file with all variables for given model
        obs_year_range = [0,np.inf]
        for i in range(len(filenames)):
            filename = filenames[i]
            var = BASEVAR[i]
            ds = xr.open_dataset(filename)
            xr_array = ds[var]
            xr_array = select_region(xr_array)
            xr_array = xr_array.sel(time=slice(TIMERANGE[0], TIMERANGE[1])) # Selects only the years specified by TIMERANGE
            if REMOVE_GLOBAL_MEAN:
                xr_array = xr_array - xr_array.mean(['lat', 'lon'],skipna=True)
            year_list = xr_array.time.dt.year.values
            year_range = [np.min(year_list),np.max(year_list)]

            # Removes partial year at the beginning or end of the dataset
            if len(year_list[year_list == year_list[-1]]) != 12: year_range[1] = year_range[1]-1
            if len(year_list[year_list == year_list[0]]) != 12: year_range[0] = year_range[0]+1
            if year_range[0] > obs_year_range[0]: obs_year_range[0] = year_range[0]
            if year_range[1] < obs_year_range[1]: obs_year_range[1] = year_range[1]
            datasets.append(xr_array)

        flat_arrays = []
        for xr_array in datasets:
            xr_array = xr_array.sel(time=slice(str(obs_year_range[0]), str(obs_year_range[1]))) #common years
            obs_years = sorted(list(set(xr_array.time.dt.year.values)))
            array = time_mean(xr_array)
            if TIMETYPE == 'month':
                flat_array = np.reshape(array, (np.shape(array)[0]//12, -1 )) #flatten if months
            else:
                flat_array = np.reshape(array, (np.shape(array)[0], -1 )) #flatten

            flat_arrays.append(flat_array) # appends flattened variable array

        obs_data = np.concatenate(flat_arrays, axis=1)

        return obs_data, np.array(obs_years)
    
    except:
        print('No observational data for this variable.')
        return None, None

# Standardizing the data
# Taking out climate sensitivity--each mean and standard deviation is taken from each individual

def detrend(data, obs=False):
    detrended = np.zeros(np.shape(data))
    x = np.arange(np.shape(data)[-2])
    if obs:
        for gp in np.arange(np.shape(data)[-1]):
            regr = linear_model.LinearRegression()
            regr.fit(x.reshape(-1, 1), data[:,gp])
            trend = regr.predict(x.reshape(-1, 1))
            detrended[:,gp] = data[:,gp] - trend + np.mean(trend)
            
        return detrended
    
    else:
        for mod in np.arange(np.shape(data)[0]):
            for gp in np.arange(np.shape(data)[-1]):
                regr = linear_model.LinearRegression()
                regr.fit(x.reshape(-1, 1), data[mod,:,gp])
                trend = regr.predict(x.reshape(-1, 1))
                detrended[mod,:,gp] = data[mod,:,gp] - trend + np.mean(trend)

        return detrended

def decade_classes(classChunk=10):
    decades = np.arange(int(TIMERANGE[0]) - 5 - classChunk,int(TIMERANGE[1]) + 10.01 + classChunk, classChunk) 
    return decades
    

def convert_fuzzyDecade(intyears,classChunk=10):
    decades = decade_classes()
    labelmap = dict()
    for d in range(len(decades)):
        labelmap[d] = decades[d]
    
    classshape = tuple([s for s in intyears.shape]+ [len(decades)])
    
    preclass = intyears.flatten()[:, np.newaxis] - decades[np.newaxis, :] # Finds difference from each decade
    preclass = np.abs(preclass)-classChunk # Years less than 10 years from the decade will be negative
    preclass[preclass>=0] = 0 # Give all other years a zero, because
    # Assigns the probabilities to each year, these are our labels for training
    classyears = (np.abs(preclass)/classChunk).reshape(classshape)

    return classyears #labelmap, labelmap.keys()

def convert_fuzzyDecade_toYear(classprobs,classChunk=10):
    decades = decade_classes()
    intyears = np.sum(decades * classprobs, axis=-1)
    
    return intyears
    
    
def standardize_data(data, years, obs = [None, None]):
    
    if str(REGION).endswith('-land'):
        maskinfo = np.load(MASK)
        boolland, boollats, boollons = maskinfo['mask'], maskinfo['lats'], maskinfo['lons']

        # Change the mask based on the region
        latrange = region_bounds[REGION]['lat']
        lonrange = region_bounds[REGION]['lon']
        boolland = boolland[np.logical_and(boollats >= latrange[0], boollats <= latrange[1]), :]
        boolland = boolland[:, np.logical_and(boollons >= lonrange[0], boollons <= lonrange[1])]
        boollons = boollats[np.logical_and(boollats >= latrange[0], boollats <= latrange[1])]
        boollons = boollons[np.logical_and(boollons >= lonrange[0], boollons <= lonrange[1])]
        
        
        #if TIMETYPE == 'month': #CHANGED
        #    boolland = np.stack([boolland]*len(VAR)*12).flatten()
        #else:
        boolland = np.stack([boolland]*len(VAR)).flatten()
        data = data[:,:,boolland]
    
    boolyr = np.logical_and(years[0]<=BASELINETIMERANGE[1], years[0] >= BASELINETIMERANGE[0]) #2019, 1980
    years_class = convert_fuzzyDecade(years)
    
    data4standardizing = data[:,boolyr,:].copy()
    data4standardizing = detrend(data4standardizing) # removes the linear trend from each data gridpoint
    
    mean = np.stack([np.mean(data4standardizing, axis=1)]*np.shape(data)[1], axis=1)
    std = np.stack([np.std(data4standardizing, axis=1)]*np.shape(data)[1], axis=1)
    std = np.mean(std, axis=(0,1)) # Average standard deviation across models at each individual grid point

    std[std == 0] = 1 # Makes sure constant points don't divide by zero
    
    data_stand = (data - mean)/std # Standardize at each lat/lon point, across time

    if 'None' not in str(obs):
        obs_data, obs_years = obs
        
        if str(REGION).endswith('-land'):
            obs_data = obs_data[:,boolland]
        
        boolyr = np.logical_and(obs_years<=BASELINETIMERANGE[1], obs_years >= BASELINETIMERANGE[0])
        
        data4standardizing = obs_data[boolyr,:].copy()
        data4standardizing = detrend(data4standardizing,obs=True) # removes the linear trend

        mean = np.nanmean(data4standardizing, axis=0) #first method
        #std = np.nanstd(data4standardizing, axis=0)
        
        obs_years_class = convert_fuzzyDecade(obs_years)
        obs_data_stand = (obs_data - mean) / std
        return data_stand, years_class, obs_data_stand, obs_years_class
    return data_stand, years_class, None, None # if observations aren't present
        
def unstandardize_years(classprobs):
    return convert_fuzzyDecade_toYear(classprobs)

def rmse(x, xi):
    return np.sqrt(np.sum((x - xi)**2/len(x)))

def mae(x, xi):
    return np.mean(np.abs(x - xi))
    
        
#####
### These functions create the npz_filename and save the netcdf data to a .npz file
#####

def make_npz_filename(directory_str):
    # make a filename like: cmip5_arrays_var=pr_tas_res=r90x45.npz
    npz_string = PRIM + '.timetype=' + TIMETYPE + '.timerange=' + TIMERANGE[0] + '_' + TIMERANGE[1] + '.var='
    
    #if TIMETYPE == 'month': #CHANGED
    #    varlist = []
    #    for var in VAR:
    #        varlist.extend( [var + '-%02i' % num for num in np.arange(12)+1] )
    #    for var in varlist:
    #        npz_string += var + '_'
    #else: 
    print(BASEVAR)
    for var in BASEVAR:
        npz_string += var + '_'
    npz_string = npz_string[:-1] 
    if REGION != None:
        npz_string += '.reg=' + REGION
    npz_string += '.globalmeanrm=' + str(REMOVE_GLOBAL_MEAN)
    npz_string += '.res=' + RES + '.npz'
    return directory_str + npz_string

def save_data_to_npz(data, years, models, lats, lons):
    # Make directories for the data (such as npzFiles_r90x45)
    new_dir = PATH + '/npzFiles_' + RES +'/'
    subprocess.run(('mkdir ' + new_dir).split())
    fn_out = make_npz_filename(new_dir)
    np.savez(fn_out, data=data, years=years, models=models, lats=lats, lons=lons)
    print('Data saved to:', fn_out)
    #data, years, models, lats, lons = [savedarrays[i] for i in ['data', 'years', 'models', 'lats', 'lons']]
    
#####
### This function opens the saved .npz files from the netcdf data
#####

def get_npz_data(filename = None):
    
    global PRIM, TIMETYPE, TIMERANGE, VAR, RES, REGION, NPZ_PATH, REMOVE_GLOBAL_MEAN, BASEVAR

    if filename == None:
        print(PRIM, TIMETYPE, RES, REMOVE_GLOBAL_MEAN, TIMERANGE, REGION, BASEVAR)
        for fn in os.listdir(NPZ_PATH):
            # Checking for correct PRIM, TIMETYPE, RES
            
            if (PRIM in fn) and ('.timetype=' + TIMETYPE in fn) and np.any('.res=' + RES in fn) \
                            and ('.globalmeanrm=' + str(REMOVE_GLOBAL_MEAN) in fn) \
                            and ('.timerange=' + TIMERANGE[0] + '_' + TIMERANGE[1] in fn):
                fn_vars = fn.split('var=')[1].split('.')[0].split('_')
                #Checking for correct REGION
                if (bool(REGION) == ('.reg=' in fn)) and (('reg=' + str(REGION) in fn.split('.')) == bool(REGION)):
                    #Checking for correct VAR
                    if BASEVAR == fn.split('var=')[1].split('.')[0].split('_'):
                        filename = fn
    else:
        PRIM = filename.split('.')[0]
        TIMETYPE = filename.split('timetype=')[1].split('.')[0]
        TIMERANGE = filename.split('timerange=')[1].split('.')[0].split('_')
        try:
            REGION = filename.split('reg=')[1].split('.')[0]
        except:
            REGION = None
        REMOVE_GLOBAL_MEAN = filename.split('globalmeanrm=')[1].split('.')[0] == 'True' #bool(True) if True
        RES = filename.split('res=')[1].split('.')[0]
        BASEVAR = filename.split('var=')[1].split('.')[0].split('_')
        NPZ_PATH = PATH + 'npzFiles_' + RES + '/'
        
    if TIMETYPE == 'month':
        varlist = []
        for var in BASEVAR:
            varlist.extend( [var + '-%02i' % num for num in np.arange(12)+1] )
        VAR = sorted(varlist)
    else:
        VAR = BASEVAR
        
    if filename == None: # One last check to see if filename is still None
        print('Error: NPZ File Not Found')
    print(filename)
    npzdata = np.load(NPZ_PATH + filename);
    return npzdata['data'], npzdata['years'], npzdata['models'], npzdata['lats'], npzdata['lons']

#####
### Stores experiments specs into a dictionary
#####
def assign_model_num():
    model_nums = [] # list of model numbers that have already been assigned
    if NEW_MODEL_NUM:
        MODEL_NUM = NEW_MODEL_NUM
        return NEW_MODEL_NUM
    for saved_model in os.listdir(ROOT + MOD_DIR):
        if saved_model.endswith('.h5'):
            model_nums.append(int(saved_model.split('.')[1])) # adds model numbers that have been used
    if len(model_nums) < 1:
        model_nums=[0]
    return max(model_nums) + 1


def track_experiment(specs, train_model_ids, val_model_ids, history=None, update=False):
    
    specs['dataset info'] = {}
    specs['dataset info']['training dataset'] = PRIM
    specs['dataset info']['variables'] = VAR
    specs['dataset info']['base variables'] = BASEVAR
    #if TIMETYPE == 'month': # CHANGED
    #    varlist = []
    #    for var in VAR:
    #        varlist.extend( [var + '-%02i' % num for num in np.arange(12)+1] )
    #    specs['dataset info']['variables'] = varlist
    specs['dataset info']['resolution'] = RES
    specs['dataset info']['region'] = REGION
    specs['dataset info']['timetype'] = TIMETYPE
    specs['dataset info']['timerange'] = TIMERANGE
    specs['dataset info']['global mean remove'] = REMOVE_GLOBAL_MEAN
    specs['dataset info']['training models'] = train_model_ids
    specs['dataset info']['validation models'] = val_model_ids
    specs['dataset info']['validation model id numbers'] = VAL_MODELS
    
    
    specs['model info'] = {}
    specs['model info']['epochs'] = EPOCHS
    specs['model info']['act func'] = ACTIVATION
    specs['model info']['ridge pen'] = RIDGE_PENALTY
    specs['model info']['lasso'] = LASSO
    specs['model info']['learn rate'] = LEARNING_RATE
    specs['model info']['model layers'] = LAYERS
    specs['model info']['random seed'] = RANDOM_NETWORK_SEED
    
    if update:
        specs['model num'] = assign_model_num()

        specs['model result'] = {}
        try:
            specs['model result']['val loss'] = '%.4f' %history.history['val_loss'][-1]
            specs['model result']['train loss'] = '%.4f' %history.history['loss'][-1]
        except:
            specs['model result']['val loss'] = None
            specs['model result']['train loss'] = None    
        
    return specs



#####
### This function splits the data into training and validation datasets
#####

def validation_split(data_stand, years_stand, model_ids):
    global VAL_MODELS
    if type(VAL_MODELS) == type(None):
        # Number of the model for the validation dataset? (array of ints)
        VAL_MODELS = np.random.choice(len(data_stand), size = round(len(data_stand)*VAL_SPLIT), replace = False)
    val_models = VAL_MODELS

    val_data = data_stand[val_models] # validation dataset uses the val_num
    val_data = np.reshape(val_data, (np.product(np.shape(val_data)[0:2]), np.shape(val_data)[2]))
    val_years = years_stand[val_models]
    val_years = val_years.reshape(np.prod(val_years.shape[0:-1]),-1)
    train_data = np.delete(data_stand, val_models, axis=0) # training dataset (validation models removed)
    train_data = np.reshape(train_data, (np.product(np.shape(train_data)[0:2]), \
                                                     np.shape(train_data)[2]))
    train_years = np.delete(years_stand, val_models, axis=0)
    train_years = train_years.reshape(np.prod(train_years.shape[0:-1]),-1)
    
    val_model_ids = model_ids[val_models]
    train_model_ids = np.delete(model_ids, val_models, axis=0)
    
    return  train_data, train_years, train_model_ids, val_data, val_years, val_model_ids,

#####
### This function defines the neural network and training
#####

# Define random seed if RANDOM_(SEGMENT/NETWORK)_SEED == None
def random_seed(random_seed):
    if random_seed == None:
        random_seed = np.random.randint(10000)
    return random_seed

def define_NN(input_shape, output_shape):
    global RANDOM_NETWORK_SEED

    model = Sequential()
    RANDOM_NETWORK_SEED = random_seed(RANDOM_NETWORK_SEED)
    
    if len(LAYERS) == 0:
        # linear model
        model.add(Dense(1, input_shape=(input_shape,), activation='linear', use_bias=True, 
                        kernel_regularizer=regularizers.l1_l2(l1=LASSO, l2=RIDGE_PENALTY),
                        bias_initializer=tf.keras.initializers.RandomNormal(seed=RANDOM_NETWORK_SEED),
                        kernel_initializer=tf.keras.initializers.RandomNormal(seed=RANDOM_NETWORK_SEED)))
    else:
        # nonlinear model - initial layer
        model.add(Dense(LAYERS[0], input_shape=(input_shape,), activation=ACTIVATION, use_bias=True, 
                        kernel_regularizer=regularizers.l1_l2(l1=LASSO, l2=RIDGE_PENALTY),
                        bias_initializer=tf.keras.initializers.RandomNormal(seed=RANDOM_NETWORK_SEED),
                        kernel_initializer=tf.keras.initializers.RandomNormal(seed=RANDOM_NETWORK_SEED)))
        # subsequent layers
        for layer in LAYERS[1:]:
            model.add(Dense(layer, activation = ACTIVATION, use_bias = True, 
                            kernel_regularizer = regularizers.l1_l2(l1=0, l2=0),
                            bias_initializer = tf.keras.initializers.RandomNormal(seed=RANDOM_NETWORK_SEED),
                            kernel_initializer = tf.keras.initializers.RandomNormal(seed=RANDOM_NETWORK_SEED)))
    
    # output layer
    model.add(Dense(output_shape, activation=None, use_bias=True, 
                    kernel_regularizer=regularizers.l1_l2(l1=0.00, l2=0.0),
                    bias_initializer=tf.keras.initializers.RandomNormal(seed=RANDOM_NETWORK_SEED),
                    kernel_initializer=tf.keras.initializers.RandomNormal(seed=RANDOM_NETWORK_SEED), 
                    kernel_constraint=tf.keras.constraints.NonNeg()))
    model.add(Activation('softmax'))
    
    return model


def train_NN(x_train, y_train, x_val, y_val):
    output_shape = len(decade_classes())
    model = define_NN(np.shape(x_train)[1], output_shape)
    
    lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
                        initial_learning_rate=LEARNING_RATE, power=1,
                        decay_steps=10000, end_learning_rate=LEARNING_RATE/20)
    
    
    #lr_schedule = LEARNING_RATE # TEMPORARY
    
    #model.compile(optimizer=optimizers.SGD(learning_rate=lr_schedule, momentum = 0.9, nesterov=True),
    #             loss = custom_loss, metrics=['mse'])
    model.compile(optimizer=optimizers.Adam(learning_rate=lr_schedule),
                 loss = 'binary_crossentropy', metrics=[tf.keras.metrics.CategoricalAccuracy()])
    history = model.fit(x_train, y_train, batch_size=32, epochs=EPOCHS, 
                        shuffle=True, verbose=0,
                        validation_data=(x_val, y_val))    
    return model, history

#####
### Save Model and Model Specifications (and also the function to retrieve them later)
#####

def check_model_exists(specs):
    global MODEL_NUM
    for modspec in os.listdir(ROOT + MOD_DIR):
        if modspec.endswith('.p'):
            f = open(ROOT + MOD_DIR + modspec,'rb')
            prev_specs = pickle.load(f) # specs from a model that was already created
            temp_prev_specs = prev_specs.copy()
            temp_specs = specs.copy()
            try: # deletes these if they already exist (but they prolly don't)
                del temp_prev_specs['model num'], temp_prev_specs['model result'], temp_specs['model num']
            except:
                pass 
            if str(temp_prev_specs) == str(temp_specs): # if the dictionary is the same as another (except model num)
                specs = prev_specs.copy() 
                MODEL_NUM = specs['model num']
                return True
    return False
                
            

def save_model(model, specs):
    model.save(ROOT + MOD_DIR + 'model.%03d.h5' % specs['model num'])
    f = open(ROOT + MOD_DIR + 'modelspecs.%03d.p' % specs['model num'],'wb')
    pickle.dump(specs,f)
    f.close()

def load_model(update_globals=True):
    # update_global updates the global variables above to fit the data the model was trained on
    model = tf.keras.models.load_model(ROOT + MOD_DIR + 'model.%03d.h5' % MODEL_NUM, compile=False)
    f = open(ROOT + MOD_DIR + 'modelspecs.%03d.p' % MODEL_NUM,'rb')
    specs = pickle.load(f)
    f.close()
    if update_globals:
        global VAR, PRIM, OBS, TIMETYPE, TIMERANGE, RES, REGION, REMOVE_GLOBAL_MEAN, VAL_MODELS, BASEVAR
        VAR = specs['dataset info']['variables']
        BASEVAR = specs['dataset info']['base variables']
        PRIM = specs['dataset info']['training dataset']
        TIMETYPE = specs['dataset info']['timetype']
        TIMERANGE = specs['dataset info']['timerange']
        RES = specs['dataset info']['resolution']
        REGION = specs['dataset info']['region']
        REMOVE_GLOBAL_MEAN = specs['dataset info']['global mean remove']
        VAL_MODELS = specs['dataset info']['validation model id numbers']
        
        global NETCDF_PATH, NPZ_PATH, SAVE_LRP_PATH, MASK
        NETCDF_PATH = PATH + 'netcdfFiles_' + RES + '/'
        NPZ_PATH = PATH + 'npzFiles_' + RES + '/' # Don't change this format
        SAVE_LRP_PATH = PATH + 'lrp_npz_' + RES + '/' + STAND_TYPE
        MASK = PATH + 'landmask/landmask_nopole_' + RES + '.npz'
        
    return model, specs
    
#####
### Plots Loss in Training
#####

def plot_error_loss(history):
    # Plot training & validation accuracy values
    plt.figure(figsize=(6,4))
    plt.plot(history.history['categorical_accuracy'])
    plt.plot(history.history['val_categorical_accuracy'])
    plt.title('Model Error')
    plt.ylabel('Categorical Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Training', 'Validation'], loc='upper left')
    #plt.ylim(0, np.max([history.history['mse'][50:], history.history['val_mse'][50:]]))
    #plt.xlim(50,EPOCHS)
    
#####
### Predict years and plot the results
#####

def plot_results(y_train, train_predict, y_val, val_predict, y_obs, obs_predict, title):
    
    plt.figure(figsize=(8,8))
    
    try:
        val_color = ['#cd5d5d','#6495ed','#dda338',
                     '#b63838','#2461cf','#9a7227',
                     '#6d2121','#143673','#584116'][MODEL_NUM-1]
        print(val_color)
    except:
        val_color = 'coral'
    
    plt.plot(y_train, train_predict, '.k', alpha=.3, markersize=10, label='Training Data')
    plt.plot(y_val, val_predict, '.', color = val_color, markersize = 10, label='Validation Data')
    
    if type(y_obs) != type(None):
        plt.plot(y_obs, obs_predict, 'o', color='black', markerfacecolor = 'white', markersize = 8, 
                 markeredgewidth=2.2, label='Observations')
    ts = 2 # textsize parameters
    minyear = np.min(y_train)
    maxyear = np.max(y_train)
    plt.plot([int(TIMERANGE[0])-5, int(TIMERANGE[1])+5], [int(TIMERANGE[0])-5, int(TIMERANGE[1])+5], 'k--')
    #plt.legend()
    tcks = [1930, 1970, 2010, 2050, 2090]
    plt.yticks(tcks, fontsize=11*ts, fontweight='demi')
    plt.xticks(tcks, fontsize=11*ts, fontweight='demi')
    plt.xlabel('Truth Year',fontsize=12*ts, fontweight='demi')
    plt.ylabel('Predicted Year',fontsize=12*ts, fontweight='demi')
    plt.title("Training: " + PRIM.upper() + ' ' + str(VAR) + ' Region=' + str(REGION), 
              size=13*ts, fontweight='demi')
    plt.grid()#axis='y')
    plt.ylim(int(TIMERANGE[0])-20,int(TIMERANGE[1])+30)
    plt.xlim(int(TIMERANGE[0])-5, int(TIMERANGE[1])+5)
    if title:
        plt.title(title, size = 13*ts, fontweight='demi')
    #plt.tight_layout()
    plt.savefig(PLOT_DIR + '/year_pred/model' + str(MODEL_NUM) + '.jpg', dpi=300, bbox_inches='tight')
    plt.show()

def predict_and_plot(model, x_train, y_train, x_val, y_val, yearmax, yearmin, datashape, obs, makeplot=True, 
                     title=None):

    y_val = unstandardize_years(y_val)
    y_train = unstandardize_years(y_train)
    train_predict = model.predict(x_train).squeeze()
    train_predict = unstandardize_years(train_predict)
    val_predict = model.predict(x_val).squeeze()
    val_predict = unstandardize_years(val_predict)
    if type(obs[0]) != type(None):
        x_obs, y_obs = obs
        y_obs = unstandardize_years(y_obs)
        obs_predict_stand = model.predict(x_obs).squeeze()
        obs_predict = unstandardize_years(obs_predict_stand)
    else:
        y_obs, obs_predict = None, None
        
    if makeplot:
        plot_results(y_train, train_predict, y_val, val_predict, y_obs, obs_predict, title)
    
    # Creating an array of predicted years that matches the shape of the original data
    pred_train_shape = datashape[0] - len(VAL_MODELS)
    pred_years = np.reshape(train_predict, (pred_train_shape, -1))
    val_pred_years = np.reshape(val_predict, (len(VAL_MODELS), -1))
    
    for i in sorted(VAL_MODELS):
        pred_years = np.insert(pred_years, i, 0, axis=0)
    for i in range(len(VAL_MODELS)):
        pred_years[VAL_MODELS[i], :] = val_pred_years[i,:]

    try:
        return pred_years, train_predict, val_predict, obs_predict
    except:
        return pred_years, train_predict, val_predict, None

def model_info(model_num=None, return_dict = False):
    if not model_num:
        model_num = MODEL_NUM
    f = open(ROOT + MOD_DIR + 'modelspecs.%03d.p' % model_num,'rb')
    specs = pickle.load(f)
    f.close()
    if return_dict:
        return specs
    print(specs)
    
def save_data_for_lrp(data_stand, obs_data_stand, years, obs_years, pred_years, lats, lons, fn_out = None):
    if not fn_out:
        fn_out = SAVE_LRP_PATH + 'lrp_model' + str(MODEL_NUM) + '.npz'
    print(np.shape(data_stand))
    np.savez(fn_out, data=data_stand, obs=obs_data_stand, years=years, 
             obs_years=obs_years, pred_years=pred_years, lats=lats, lons=lons)
    print('Saved to LRP at ' + fn_out)
    
def save_pred_years(pred_years, fn_out = None):
    if fn_out:
        np.save(fn_out, pred_years)
        print('Saved predicted years to ' + fn_out)
    else:
        "Predicted Years Not Saved!!!"

# Let's try a bunch of sets of models

In [50]:
#all_val_models = list(range(36))
#random_val_list = []
#for i in range(100):
#    random.shuffle(all_val_models)
#    random_val_list.append(all_val_models[:7])

In [11]:
TRIALTYPE = '_' + REGION
print(TRIALTYPE)

## NEED TO DO THIS!
EXPERIMENT_NAME = 'pr_tas_shifted' + TRIALTYPE +  '/'
STAND_TYPE = 'classification_topmodels/'
SAVE_LRP_PATH = PATH + 'lrp_npz_' + RES + '/' + STAND_TYPE + EXPERIMENT_NAME
NUM_MODELS = 100

allvars = [sorted(['tas']), sorted(['pr']), sorted(['tas', 'pr']), 
           sorted(['tas']), sorted(['pr']), sorted(['tas', 'pr']),
           sorted(['tas-djf', 'tas-mam', 'tas-jja', 'tas-son']),
           sorted(['pr-djf', 'pr-mam', 'pr-jja', 'pr-son']),
           sorted(['tas-djf', 'tas-mam', 'tas-jja', 'tas-son','pr-djf', 'pr-mam', 'pr-jja', 'pr-son']),
           sorted(['tas-fma', 'tas-mjj', 'tas-aso', 'tas-ndj']),
           sorted(['pr-fma', 'pr-mjj', 'pr-aso', 'pr-ndj']), 
           sorted(['tas-fma', 'tas-mjj', 'tas-aso', 'tas-ndj','pr-fma', 'pr-mjj', 'pr-aso', 'pr-ndj']),
           sorted(['tas-jfm', 'tas-amj', 'tas-jas', 'tas-ond']),
           sorted(['pr-jfm', 'pr-amj', 'pr-jas', 'pr-ond']), 
           sorted(['tas-jfm', 'tas-amj', 'tas-jas', 'tas-ond','pr-jfm', 'pr-amj', 'pr-jas', 'pr-ond']),           
          ]  

allvars = [sorted(['cdd-jfm', 'cdd-amj', 'cdd-jas', 'cdd-ond', 'hdd-jfm', 'hdd-amj', 'hdd-jas', 'hdd-ond']),
           sorted(['cdd-djf', 'cdd-mam', 'cdd-jja', 'cdd-son', 'hdd-djf', 'hdd-mam', 'hdd-jja', 'hdd-son']),
           sorted(['cdd-fma', 'cdd-mjj', 'cdd-aso', 'cdd-ndj', 'hdd-fma', 'hdd-mjj', 'hdd-aso', 'hdd-ndj'])]

allvars = [sorted(['r95p-jfm', 'r95p-amj', 'r95p-jas', 'r95p-ond']),
           sorted(['r95p-djf', 'r95p-mam', 'r95p-jja', 'r95p-son']),
           sorted(['r95p-fma', 'r95p-mjj', 'r95p-aso', 'r95p-ndj']),
           sorted(['r95p'])]

allvars = [sorted(['tas-djf', 'tas-mam', 'tas-jja', 'tas-son','pr-djf', 'pr-mam', 'pr-jja', 'pr-son'])]

variable_names = ['tas-month', 'pr-month', 'pr_tas-month',
                  'tas-ann', 'pr-ann', 'pr_tas-ann',
                  'tas-seas-met', 'pr-seas-met', 'pr_tas-seas-met',
                 'tas-seas-alt', 'pr-seas-alt', 'pr_tas-seas-alt',
                 'tas-seas-ord', 'pr-seas-ord', 'pr_tas-seas-ord']

variable_names = ['dd-month_1', 'dd-ann_1', 'dd-seas-ord_1', 'dd-month_.1', 'dd-ann_.1', 'dd-seas-ord_.1',
                  'dd-month_.01', 'dd-ann_.01', 'dd-seas-ord_.01']

variable_names = ['pr_tas-seas-met-shiftedbaseline']
 

timetype = ['month', 'month', 'month', 
            'annual', 'annual', 'annual',
            'annual', 'annual', 'annual',
            'annual', 'annual', 'annual',
            'annual', 'annual', 'annual',]

timetype = ['annual'] * 4

timetype = ['annual']

ridges = [.1,.1,.1,
          .1, .1, .1,
          .1, .1, .01, 
          .1, .1, .01, 
          .1, .1, .01]

ridges = [1, 1, 1, .1, .1, .1, .01, .01, .01]

ridges = [.1] * 4

ridges = [.01]

nummodels = 37 #37 (taspr) #32 (r95p) #34 (dd)

# OPening prior dictionary
try:
    print(ROOT + '/playground_data/' + EXPERIMENT_NAME + 'bestmodeldictionary_cmip6_classification'\
             +TRIALTYPE+'.p')
    f = open(ROOT + '/playground_data/' + EXPERIMENT_NAME + 'bestmodeldictionary_cmip6_classification'\
             +TRIALTYPE+'.p', 'rb')
    findbestmodels = pickle.load(f)
    f.close()
    findbestmodelsold = findbestmodels.copy()
    valmodelsspecified = True
    print('Opening Established Dictionary.')
    print('Current Keys: ', findbestmodels.keys())
except:
    findbestmodels = dict()
    os.system('mkdir ' + ROOT + '/playground_data/' + EXPERIMENT_NAME)
    os.system('mkdir ' + SAVE_LRP_PATH)
    os.system('mkdir ' + SAVE_LRP_PATH + 'predyears/')
    print('No dictionary found in files, starting new dictionary.')
    valmodelsspecified = False
    
minimumkeys = len(findbestmodels.keys())
print(minimumkeys)

## Which models will you be training on?
if valmodelsspecified:
    f = open(ROOT + '/playground_data/' + EXPERIMENT_NAME + 'validation_models_list.p','rb')
    random_val_list = pickle.load(f)
    print('Loading validation models for ' + EXPERIMENT_NAME)
else:
    print('Generating New Validation Models.')
    all_val_models = list(range(nummodels))
    random_val_list = []
    for i in range(NUM_MODELS):
        random.shuffle(all_val_models)
        random_val_list.append(all_val_models[:nummodels//5])
    f = open(ROOT + '/playground_data/' + EXPERIMENT_NAME + 'validation_models_list.p','wb')
    pickle.dump(random_val_list, f)
    f.close()


for BASEVAR, RIDGE_PENALTY, longname, TIMETYPE in zip(allvars, ridges, variable_names, timetype):    
    
    if TIMETYPE == 'month':
        varlist = []
        for var in BASEVAR:
            varlist.extend( [var + '-%02i' % num for num in np.arange(12)+1] )
        VAR = sorted(varlist)
    else:
        VAR = BASEVAR
    
    KEY = str(VAR)
    
    if KEY not in findbestmodels.keys():
        print(KEY)

        findbestmodels[KEY] = dict()

        data, years, model_ids, lats, lons = prepare_data()
        save_data_to_npz(data, years, model_ids, lats, lons)

        # Getting the data from NPZ
        data, years, model_ids, lats, lons = get_npz_data() # Gets training data
        #obs_data, obs_years = get_obs() # Gets observational data, prepared same as training data
        obs_data, obs_years = None, None

        # Now standardizing the data
        data_stand, years_stand, obs_data_stand, obs_years_stand \
            = standardize_data(data, years, obs = (obs_data, obs_years))

        for iteration in np.arange(NUM_MODELS):

            VAL_MODELS = random_val_list[iteration]



            # splitting the standardized data into validation and training datasets
            train_data, train_years, train_model_ids, val_data, val_years, val_model_ids\
                = validation_split(data_stand, years_stand, model_ids) 

            train_years_raw = unstandardize_years(train_years)
            val_years_raw = unstandardize_years(val_years)

            # Start experiment tracking
            specs_dict = track_experiment({}, train_model_ids, val_model_ids)

            # Training the Neural Network (if the model doesn't exist, otherwise it updates the specs_dict)
            try:
                
                # Loading the model
                model = tf.keras.models.load_model(ROOT + '/playground_data/' + EXPERIMENT_NAME + \
                                                   longname + '_models/model.%03d.h5' % iteration, compile=False)
                print('Model exists. Loading Model.', iteration, VAR)
                print(ROOT + '/playground_data/' + EXPERIMENT_NAME + longname + '_models/model.%03d.h5' % iteration)
                # Plotting the predicted years against the truth years
                yearmax, yearmin = int(TIMERANGE[1]), int(TIMERANGE[0])
                pred_years, train_predict, val_predict, obs_predict = \
                            predict_and_plot(model, train_data, train_years, val_data, val_years, yearmax, 
                                             yearmin, np.shape(data_stand),
                                             obs = [obs_data_stand, obs_years_stand], makeplot=False)
            except:
                print('New experiment. Training ANN.', iteration, VAR)
                model, history = train_NN(train_data, train_years, val_data, val_years)

            # Updates tracking of the experiment, also assigns an integer ID for the model
            try:
                specs_dict = track_experiment(specs_dict, train_model_ids, val_model_ids, history=history, update=True)
            except:
                specs_dict = track_experiment(specs_dict, train_model_ids, val_model_ids, history=None, update=True)

            # Plotting the predicted years against the truth years
            yearmax, yearmin = int(TIMERANGE[1]), int(TIMERANGE[0])
            pred_years, train_predict, val_predict, obs_predict = \
                                          predict_and_plot(model, train_data, train_years, val_data, val_years, yearmax, 
                                                           yearmin, np.shape(data_stand),
                                                           obs = [obs_data_stand, obs_years_stand], makeplot=False)

            # updating experiment to include errors for training, validation, and observational data

            try:
                specs_dict['model result']['obs rmse'] = '%.2f' %rmse(obs_years[obs_years >=1980], 
                                                                      obs_predict[obs_years >=1980])
                specs_dict['model result']['obs mae'] = '%.2f' %mae(obs_years[obs_years >=1980], 
                                                                    obs_predict[obs_years >=1980])
                specs_dict['model result']['obs corr'] = '%.3f' %ss.pearsonr(obs_years[obs_years >=1980], 
                                                                             obs_predict[obs_years >=1980])[0]
                # Observations metrics relative to the training data
                train_predict_mean = np.array([np.mean(train_predict[train_years_raw==uyr]) for uyr in sorted(np.unique(obs_years))])
                specs_dict['model result']['obs rmse (relative to train)'] = '%.2f' %rmse(obs_years[obs_years >=1980], 
                                                                      train_predict_mean[obs_years >=1980])
                specs_dict['model result']['obs mae (relative to train)'] = '%.2f' %mae(obs_years[obs_years >=1980], 
                                                                    train_predict_mean[obs_years >=1980])
                specs_dict['model result']['obs corr (relative to train)'] = '%.3f' %ss.pearsonr(obs_years[obs_years >=1980], 
                                                                             train_predict_mean[obs_years >=1980])[0]
            except:
                specs_dict['model result']['obs rmse'] = None
                specs_dict['model result']['obs mae'] = None
                specs_dict['model result']['obs corr'] = None
                specs_dict['model result']['obs rmse (relative to train)'] = None
                specs_dict['model result']['obs mae (relative to train)'] = None
                specs_dict['model result']['obs corr (relative to train)'] = None

            specs_dict['model result']['train rmse'] = '%.2f' %rmse(train_years_raw, train_predict)
            specs_dict['model result']['train mae'] = '%.2f' %mae(train_years_raw, train_predict)

            specs_dict['model result']['val rmse'] = '%.2f' %rmse(val_years_raw, val_predict)
            specs_dict['model result']['val mae'] = '%.2f' %mae(val_years_raw, val_predict)

            specs_dict['model result']['train rmse pre-2020'] = '%.2f' %rmse(train_years_raw[train_years_raw < 2020], 
                                                                             train_predict[train_years_raw < 2020])
            specs_dict['model result']['val rmse pre-2020'] = '%.2f' %rmse(val_years_raw[val_years_raw < 2020], 
                                                                           val_predict[val_years_raw < 2020])

            specs_dict['model result']['train rmse post-2060'] = '%.2f' %rmse(train_years_raw[train_years_raw > 2060], 
                                                                             train_predict[train_years_raw > 2060])
            specs_dict['model result']['val rmse post-2060'] = '%.2f' %rmse(val_years_raw[val_years_raw > 2060], 
                                                                           val_predict[val_years_raw > 2060])

            if TIMERANGE == ['1920','2099'] or TIMERANGE == ['1920','2098']:
                # Calculate TOE using 1920-1960 values
                train_predict_array = train_predict.reshape(-1,int(TIMERANGE[1]) - int(TIMERANGE[0]) + 1)
                val_predict_array = val_predict.reshape(-1,int(TIMERANGE[1]) - int(TIMERANGE[0]) + 1)

                train_baselinemax = np.max(train_predict_array[:, :40], axis=1).reshape(-1,1)
                val_baselinemax = np.max(val_predict_array[:, :40], axis=1).reshape(-1,1)
                train_toe_bool = train_predict_array < train_baselinemax
                val_toe_bool = val_predict_array < val_baselinemax

                # TOE is the year one step beyond the last year where 95 CIs overlap between a given year and 1920-1959
                uniq_years_reverse = np.flip(np.unique(train_years_raw))
                # last year that doesn't exceed 1920-1960 values plus one
                train_toe_years = uniq_years_reverse[np.argmax(np.flip(train_toe_bool,axis=1),axis=1)]+1 
                val_toe_years = uniq_years_reverse[np.argmax(np.flip(val_toe_bool,axis=1),axis=1)]+1 


                specs_dict['model result']['TOE(val)'] = val_toe_years.astype(int)
                specs_dict['model result']['TOE(train)'] = train_toe_years.astype(int)
                specs_dict['model result']['TOE(val mean)'] = np.round(np.mean(val_toe_years),decimals=1)
              
            # Save the data for LRP
            if iteration == 0:
                save_data_for_lrp(data_stand, obs_data_stand, years, obs_years, pred_years, lats, lons,
                              fn_out = SAVE_LRP_PATH + 'lrp_' + longname + '.npz')
            os.system('mkdir ' + SAVE_LRP_PATH + 'predyears/')
            save_pred_years(pred_years, fn_out = SAVE_LRP_PATH + 'predyears/predyears_' + longname + '_%03d.npy' % iteration)
                
            # Printing the specs_dict
            findbestmodels[KEY][iteration] = specs_dict
            
            # Save each individual model
            os.system('mkdir ' + ROOT + '/playground_data/' + EXPERIMENT_NAME)
            os.system('mkdir ' + ROOT + '/playground_data/' + EXPERIMENT_NAME + longname + '_models/')
            model.save(ROOT + '/playground_data/' + EXPERIMENT_NAME + longname + '_models/model.%03d.h5' % iteration)

        # Save the dictionary at it's current checkpoint
        f = open(ROOT + '/playground_data/' + EXPERIMENT_NAME + \
                 'bestmodeldictionary_cmip6_classification'+TRIALTYPE+'.p', 'wb')
        pickle.dump(findbestmodels, f)
        f.close()
        
    else:
        print('Variable already exists in dictionary.')

        """data, years, model_ids, lats, lons = prepare_data()
        save_data_to_npz(data, years, model_ids, lats, lons)

        # Getting the data from NPZ
        data, years, model_ids, lats, lons = get_npz_data() # Gets training data
        obs_data, obs_years = get_obs() # Gets observational data, prepared same as training data

        # Now standardizing the data
        data_stand, years_stand, obs_data_stand, obs_years_stand \
            = standardize_data(data, years, obs = (obs_data, obs_years))

        for iteration in np.arange(NUM_MODELS):

            VAL_MODELS = random_val_list[iteration]



            # splitting the standardized data into validation and training datasets
            train_data, train_years, train_model_ids, val_data, val_years, val_model_ids\
                = validation_split(data_stand, years_stand, model_ids) 

            train_years_raw = unstandardize_years(train_years)
            val_years_raw = unstandardize_years(val_years)
        
            # Loading the model
            model = tf.keras.models.load_model(ROOT + '/playground_data/' + EXPERIMENT_NAME + longname + '_models/model.%03d.h5' % iteration, compile=False)
            print('Model exists. Loading Model.', iteration, VAR)
            print(ROOT + '/playground_data/' + EXPERIMENT_NAME + longname + '_models/model.%03d.h5' % iteration)
            # Plotting the predicted years against the truth years
            yearmax, yearmin = int(TIMERANGE[1]), int(TIMERANGE[0])
            pred_years, train_predict, val_predict, obs_predict = \
                        predict_and_plot(model, train_data, train_years, val_data, val_years, yearmax, 
                                         yearmin, np.shape(data_stand),
                                         obs = [obs_data_stand, obs_years_stand], makeplot=False)
            
            os.system('mkdir ' + SAVE_LRP_PATH + 'predyears/')
            save_pred_years(pred_years, fn_out = SAVE_LRP_PATH + 'predyears/predyears_' + longname + '_%03d.npy' % iteration)
            """
# This is unimportant if already saving the dictionary at each check point    
#f = open(ROOT + '/playground_data/' + EXPERIMENT_NAME + \
#         'bestmodeldictionary_cmip6_classification'+TRIALTYPE+'.p', 'wb')
#pickle.dump(findbestmodels, f)
#f.close()

_globe-land
/Users/jrader/Rams/TOE_Research//playground_data/pr_tas_shifted_globe-land/bestmodeldictionary_cmip6_classification_globe-land.p
Opening Established Dictionary.
Current Keys:  dict_keys(["['pr-djf', 'pr-jja', 'pr-mam', 'pr-son', 'tas-djf', 'tas-jja', 'tas-mam', 'tas-son']"])
1
Loading validation models for pr_tas_shifted_globe-land/
Variable already exists in dictionary.


In [21]:
f = open(ROOT + '/playground_data/' + EXPERIMENT_NAME + \
         'bestmodeldictionary_cmip6_classification'+TRIALTYPE+'.p', 'wb')
pickle.dump(findbestmodels, f)
f.close()

In [28]:
findbestmodels.keys()

dict_keys(["['cdd-01', 'cdd-02', 'cdd-03', 'cdd-04', 'cdd-05', 'cdd-06', 'cdd-07', 'cdd-08', 'cdd-09', 'cdd-10', 'cdd-11', 'cdd-12', 'hdd-01', 'hdd-02', 'hdd-03', 'hdd-04', 'hdd-05', 'hdd-06', 'hdd-07', 'hdd-08', 'hdd-09', 'hdd-10', 'hdd-11', 'hdd-12']_1", "['cdd', 'hdd']_1", "['cdd-amj', 'cdd-jas', 'cdd-jfm', 'cdd-ond', 'hdd-amj', 'hdd-jas', 'hdd-jfm', 'hdd-ond']_1", "['cdd-01', 'cdd-02', 'cdd-03', 'cdd-04', 'cdd-05', 'cdd-06', 'cdd-07', 'cdd-08', 'cdd-09', 'cdd-10', 'cdd-11', 'cdd-12', 'hdd-01', 'hdd-02', 'hdd-03', 'hdd-04', 'hdd-05', 'hdd-06', 'hdd-07', 'hdd-08', 'hdd-09', 'hdd-10', 'hdd-11', 'hdd-12']_0.1", "['cdd', 'hdd']_0.1", "['cdd-amj', 'cdd-jas', 'cdd-jfm', 'cdd-ond', 'hdd-amj', 'hdd-jas', 'hdd-jfm', 'hdd-ond']_0.1", "['cdd-01', 'cdd-02', 'cdd-03', 'cdd-04', 'cdd-05', 'cdd-06', 'cdd-07', 'cdd-08', 'cdd-09', 'cdd-10', 'cdd-11', 'cdd-12', 'hdd-01', 'hdd-02', 'hdd-03', 'hdd-04', 'hdd-05', 'hdd-06', 'hdd-07', 'hdd-08', 'hdd-09', 'hdd-10', 'hdd-11', 'hdd-12']_0.01", "['cdd', 'hdd'

In [20]:
findbestmodels.keys()

dict_keys(["['tas-djf', 'tas-jja', 'tas-mam', 'tas-son']", "['pr-djf', 'pr-jja', 'pr-mam', 'pr-son']", "['tas-aso', 'tas-fma', 'tas-mjj', 'tas-ndj']", "['pr-aso', 'pr-fma', 'pr-mjj', 'pr-ndj']", "['tas-amj', 'tas-jas', 'tas-jfm', 'tas-ond']", "['pr-amj', 'pr-jas', 'pr-jfm', 'pr-ond']"])

In [23]:
findbestmodels.keys()


dict_keys(["['tas-01', 'tas-02', 'tas-03', 'tas-04', 'tas-05', 'tas-06', 'tas-07', 'tas-08', 'tas-09', 'tas-10', 'tas-11', 'tas-12']", "['pr-01', 'pr-02', 'pr-03', 'pr-04', 'pr-05', 'pr-06', 'pr-07', 'pr-08', 'pr-09', 'pr-10', 'pr-11', 'pr-12', 'tas-01', 'tas-02', 'tas-03', 'tas-04', 'tas-05', 'tas-06', 'tas-07', 'tas-08', 'tas-09', 'tas-10', 'tas-11', 'tas-12']", "['pr-01', 'pr-02', 'pr-03', 'pr-04', 'pr-05', 'pr-06', 'pr-07', 'pr-08', 'pr-09', 'pr-10', 'pr-11', 'pr-12']", "['tas-djf', 'tas-jja', 'tas-mam', 'tas-son']", "['pr-djf', 'pr-jja', 'pr-mam', 'pr-son']", "['pr-djf', 'pr-jja', 'pr-mam', 'pr-son', 'tas-djf', 'tas-jja', 'tas-mam', 'tas-son']", "['tas-aso', 'tas-fma', 'tas-mjj', 'tas-ndj']", "['pr-aso', 'pr-fma', 'pr-mjj', 'pr-ndj']", "['pr-aso', 'pr-fma', 'pr-mjj', 'pr-ndj', 'tas-aso', 'tas-fma', 'tas-mjj', 'tas-ndj']", "['tas-amj', 'tas-jas', 'tas-jfm', 'tas-ond']", "['pr-amj', 'pr-jas', 'pr-jfm', 'pr-ond']", "['pr-amj', 'pr-jas', 'pr-jfm', 'pr-ond', 'tas-amj', 'tas-jas', 'tas-

In [30]:
decade_classes()

array([1905., 1915., 1925., 1935., 1945., 1955., 1965., 1975., 1985.,
       1995., 2005., 2015., 2025., 2035., 2045., 2055., 2065., 2075.,
       2085., 2095., 2105.])

# Updating the dictionaries with metrics

In [4]:
TRIALTYPE = '_' + REGION
print(TRIALTYPE)
## NEED TO DO THIS!
EXPERIMENT_NAME = 'pr_tas_final' + TRIALTYPE + '/'
allvars = [sorted(['tas']), sorted(['tas', 'pr']), sorted(['pr']), 
           sorted(['tas']), sorted(['tas', 'pr']), sorted(['pr']),
           sorted(['tas-djf', 'tas-mam', 'tas-jja', 'tas-son']),
           sorted(['pr-djf', 'pr-mam', 'pr-jja', 'pr-son']),
           sorted(['pr-djf', 'pr-mam', 'pr-jja', 'pr-son','tas-djf', 'tas-mam', 'tas-jja', 'tas-son']),
           
          ]       

variable_names = ['tas-ann', 'pr_tas-ann', 'pr-ann',
                  'tas-month', 'pr_tas-month', 'pr-month',
                  'tas-seas-met', 'pr-seas-met', 'pr_tas-seas-met']

timetype = ['annual', 'annual', 'annual',
            'month', 'month', 'month',
            'annual', 'annual', 'annual']

nummodels = 37

STAND_TYPE = 'classification_topmodels/'
SAVE_LRP_PATH = PATH + 'lrp_npz_' + RES + '/' + STAND_TYPE + EXPERIMENT_NAME

# OPening prior dictionary
try:
    print(ROOT + '/playground_data/' + EXPERIMENT_NAME + 'bestmodeldictionary_cmip6_classification'\
             +TRIALTYPE+'.p')
    f = open(ROOT + '/playground_data/' + EXPERIMENT_NAME + 'bestmodeldictionary_cmip6_classification'\
             +TRIALTYPE+'.p', 'rb')
    findbestmodels = pickle.load(f)
    f.close()
    findbestmodelsold = findbestmodels.copy()
    valmodelsspecified = True
    print('Opening Established Dictionary.')
    print('Current Keys: ', findbestmodels.keys())
except:
    findbestmodelsold = dict()
    valmodelsspecified = True

## Which models will you be training on?
if valmodelsspecified:
    f = open(ROOT + '/playground_data/' + EXPERIMENT_NAME + 'validation_models_list.p','rb')
    random_val_list = pickle.load(f)
    print('Loading validation models for ' + EXPERIMENT_NAME)
else:
    print('Generating New Validation Models.')
    import nada
    all_val_models = list(range(nummodels))
    random_val_list = []
    for i in range(100):
        random.shuffle(all_val_models)
        random_val_list.append(all_val_models[:nummodels//5])

for BASEVAR, longname, TIMETYPE in zip(allvars, variable_names, timetype):  
        # Getting the data from NPZ
        data, years, model_ids, lats, lons = get_npz_data() # Gets training data
        obs_data, obs_years = get_obs() # Gets observational data, prepared same as training data
        
        # Now standardizing the data
        data_stand, years_stand, obs_data_stand, obs_years_stand \
            = standardize_data(data, years, obs = (obs_data, obs_years))

        for iteration in np.arange(100):

            VAL_MODELS = random_val_list[iteration]
            
            # splitting the standardized data into validation and training datasets
            train_data, train_years, train_model_ids, val_data, val_years, val_model_ids\
                = validation_split(data_stand, years_stand, model_ids) 

            train_years_raw = unstandardize_years(train_years)
            val_years_raw = unstandardize_years(val_years)

            # Loading the model
            model = tf.keras.models.load_model(ROOT + '/playground_data/' + EXPERIMENT_NAME + longname + '_models/model.%03d.h5' % iteration, compile=False)
            print(ROOT + '/playground_data/' + EXPERIMENT_NAME + longname + '_models/model.%03d.h5' % iteration)
            # Plotting the predicted years against the truth years
            yearmax, yearmin = int(TIMERANGE[1]), int(TIMERANGE[0])
            pred_years, train_predict, val_predict, obs_predict = \
                        predict_and_plot(model, train_data, train_years, val_data, val_years, yearmax, 
                                         yearmin, np.shape(data_stand),
                                         obs = [obs_data_stand, obs_years_stand], makeplot=False)
            # Save data out for doing LRP
            if iteration == 0:
                save_data_for_lrp(data_stand, obs_data_stand, years, obs_years, pred_years, lats, lons,
                              fn_out = SAVE_LRP_PATH + 'lrp_' + longname + '.npz')

                
            # Calculate TOE using 1920-1960 values THIS IS FOR 5-year RUNNING
            train_predict_array1 = train_predict.reshape(-1,int(TIMERANGE[1]) - int(TIMERANGE[0]) + 1)
            val_predict_array1 = val_predict.reshape(-1,int(TIMERANGE[1]) - int(TIMERANGE[0]) + 1)

            # Do a 5 year running mean
            N = 5
            val_predict_array = np.zeros((val_predict_array1.shape[0], val_predict_array1.shape[1] - N + 1))
            for i in range(val_predict_array.shape[0]): 
                val_predict_array[i, :] = np.convolve(val_predict_array1[i, :], np.ones(N)/N, mode='valid')
                
            train_predict_array = np.zeros((train_predict_array1.shape[0], train_predict_array1.shape[1] - N + 1))
            for i in range(train_predict_array.shape[0]): 
                train_predict_array[i, :] = np.convolve(train_predict_array1[i, :], np.ones(N)/N, mode='valid')
            
            train_baselinemax = np.max(train_predict_array[:, :36], axis=1).reshape(-1,1)
            val_baselinemax = np.max(val_predict_array[:, :36], axis=1).reshape(-1,1)
            train_toe_bool = train_predict_array < train_baselinemax
            val_toe_bool = val_predict_array < val_baselinemax

            # TOE is the year one step beyond the last year where 95 CIs overlap between a given year and 1920-1959
            uniq_years_reverse = np.flip(np.unique(train_years_raw[4:])) # Losing 4 years because of 5-yr mean
            # last year that doesn't exceed 1920-1960 values plus one
            train_toe_years = uniq_years_reverse[np.argmax(np.flip(train_toe_bool,axis=1),axis=1)]+1 
            val_toe_years = uniq_years_reverse[np.argmax(np.flip(val_toe_bool,axis=1),axis=1)]+1 

            findbestmodels[str(VAR)][iteration]['model result']['TOE(5-yr val)'] = val_toe_years.astype(int)
            findbestmodels[str(VAR)][iteration]['model result']['TOE(5-yr train)'] = train_toe_years.astype(int)
            findbestmodels[str(VAR)][iteration]['model result']['TOE(5-yr val mean)'] = np.round(np.mean(val_toe_years),decimals=1)
                
                
#             # Add observations metrics
#             obsrmse = '%.2f' %rmse(obs_years[obs_years >=1980], 
#                                                                   obs_predict[obs_years >=1980])
#             obsmae = '%.2f' %mae(obs_years[obs_years >=1980], 
#                                                                 obs_predict[obs_years >=1980])
#             obscorr = '%.3f' %ss.pearsonr(obs_years[obs_years >=1980], 
#                                                                          obs_predict[obs_years >=1980])[0]
            
#             # Observations metrics relative to the training data
#             train_predict_mean = np.array([np.mean(train_predict[train_years_raw==uyr]) for uyr in sorted(np.unique(obs_years))])
#             obsrmset = '%.2f' %rmse(obs_years[obs_years >=1980], 
#                                                                   train_predict_mean[obs_years >=1980])
#             obsmaet = '%.2f' %mae(obs_years[obs_years >=1980], 
#                                                                 train_predict_mean[obs_years >=1980])
#             obscorrt = '%.3f' %ss.pearsonr(obs_years[obs_years >=1980], 
#                                                                          train_predict_mean[obs_years >=1980])[0]
            
#             # Calculate observations metrics
#             findbestmodels[str(VAR)][iteration]['model result']['obs rmse (' + OBS['pr'].lower() + ')'] = obsrmse
#             findbestmodels[str(VAR)][iteration]['model result']['obs mae (' + OBS['pr'].lower() + ')'] = obsmae
#             findbestmodels[str(VAR)][iteration]['model result']['obs corr (' + OBS['pr'].lower() + ')'] = obscorr
#             findbestmodels[str(VAR)][iteration]['model result']['obs rmse (relative to train) (' + OBS['pr'].lower() + ')'] = obsrmset
#             findbestmodels[str(VAR)][iteration]['model result']['obs mae (relative to train) (' + OBS['pr'].lower() + ')'] = obsmaet
#             findbestmodels[str(VAR)][iteration]['model result']['obs corr (relative to train) (' + OBS['pr'].lower() + ')'] = obscorrt
                
            
f = open(ROOT + '/playground_data/' + EXPERIMENT_NAME + \
         'bestmodeldictionary_cmip6_classification'+TRIALTYPE+'.p', 'wb')
pickle.dump(findbestmodels, f)
f.close()

_globe-land
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/bestmodeldictionary_cmip6_classification_globe-land.p
Opening Established Dictionary.
Current Keys:  dict_keys(["['tas-djf', 'tas-jja', 'tas-mam', 'tas-son']", "['pr-djf', 'pr-jja', 'pr-mam', 'pr-son']", "['tas-aso', 'tas-fma', 'tas-mjj', 'tas-ndj']", "['pr-aso', 'pr-fma', 'pr-mjj', 'pr-ndj']", "['tas-amj', 'tas-jas', 'tas-jfm', 'tas-ond']", "['pr-amj', 'pr-jas', 'pr-jfm', 'pr-ond']", "['tas']", "['pr']", "['pr', 'tas']", "['tas-01', 'tas-02', 'tas-03', 'tas-04', 'tas-05', 'tas-06', 'tas-07', 'tas-08', 'tas-09', 'tas-10', 'tas-11', 'tas-12']", "['pr-01', 'pr-02', 'pr-03', 'pr-04', 'pr-05', 'pr-06', 'pr-07', 'pr-08', 'pr-09', 'pr-10', 'pr-11', 'pr-12']", "['pr-01', 'pr-02', 'pr-03', 'pr-04', 'pr-05', 'pr-06', 'pr-07', 'pr-08', 'pr-09', 'pr-10', 'pr-11', 'pr-12', 'tas-01', 'tas-02', 'tas-03', 'tas-04', 'tas-05', 'tas-06', 'tas-07', 'tas-08', 'tas-09', 'tas-10', 'tas-11', 'tas-12']", "['pr-djf', 'pr-jja', 

/Users/Jamin/anaconda3/envs/toe2/lib/python3.7/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/tas-ann_models/model.000.h5
(37, 180, 948)
Saved to LRP at /Users/Jamin/Rams/TOE_Research/data/lrp_npz_r90x45/classification_topmodels/pr_tas_final_globe-land/lrp_tas-ann.npz
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/tas-ann_models/model.001.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/tas-ann_models/model.002.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/tas-ann_models/model.003.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/tas-ann_models/model.004.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/tas-ann_models/model.005.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/tas-ann_models/model.006.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/tas-ann_models/model.007.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_ta

/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/tas-ann_models/model.081.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/tas-ann_models/model.082.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/tas-ann_models/model.083.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/tas-ann_models/model.084.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/tas-ann_models/model.085.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/tas-ann_models/model.086.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/tas-ann_models/model.087.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/tas-ann_models/model.088.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/tas-ann_models/model.089.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/tas-ann_models/model.090.h5


/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr_tas-ann_models/model.058.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr_tas-ann_models/model.059.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr_tas-ann_models/model.060.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr_tas-ann_models/model.061.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr_tas-ann_models/model.062.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr_tas-ann_models/model.063.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr_tas-ann_models/model.064.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr_tas-ann_models/model.065.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr_tas-ann_models/model.066.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/p

...

/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/tas-seas-met_models/model.077.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/tas-seas-met_models/model.078.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/tas-seas-met_models/model.079.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/tas-seas-met_models/model.080.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/tas-seas-met_models/model.081.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/tas-seas-met_models/model.082.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/tas-seas-met_models/model.083.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/tas-seas-met_models/model.084.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/tas-seas-met_models/model.085.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_

/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr-seas-met_models/model.052.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr-seas-met_models/model.053.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr-seas-met_models/model.054.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr-seas-met_models/model.055.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr-seas-met_models/model.056.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr-seas-met_models/model.057.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr-seas-met_models/model.058.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr-seas-met_models/model.059.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr-seas-met_models/model.060.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_glo

/Users/Jamin/anaconda3/envs/toe2/lib/python3.7/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr_tas-seas-met_models/model.000.h5
(37, 180, 7584)
Saved to LRP at /Users/Jamin/Rams/TOE_Research/data/lrp_npz_r90x45/classification_topmodels/pr_tas_final_globe-land/lrp_pr_tas-seas-met.npz
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr_tas-seas-met_models/model.001.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr_tas-seas-met_models/model.002.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr_tas-seas-met_models/model.003.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr_tas-seas-met_models/model.004.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr_tas-seas-met_models/model.005.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr_tas-seas-met_models/model.006.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr_tas-seas-met_

/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr_tas-seas-met_models/model.075.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr_tas-seas-met_models/model.076.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr_tas-seas-met_models/model.077.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr_tas-seas-met_models/model.078.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr_tas-seas-met_models/model.079.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr_tas-seas-met_models/model.080.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr_tas-seas-met_models/model.081.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr_tas-seas-met_models/model.082.h5
/Users/Jamin/Rams/TOE_Research//playground_data/pr_tas_final_globe-land/pr_tas-seas-met_models/model.083.h5
/Users/Jamin/Rams/TOE_Resear

In [18]:
val_predict_array.shape

(7, 180)

In [8]:
#f = open(ROOT + '/playground_data/' + EXPERIMENT_NAME + \
#         'bestmodeldictionary_cmip6_classification'+TRIALTYPE+'.p', 'wb')
#pickle.dump(findbestmodels, f)
#f.close()

In [16]:
180 * 30

5400

In [22]:
N = 5
pred5 = np.zeros((val_predict_array.shape[0], val_predict_array.shape[1] - N + 1))
for i in range(val_predict_array.shape[0]): 
    pred5[i, :] = np.convolve(val_predict_array[i, :], np.ones(N)/N, mode='valid')

In [25]:
np.any(pred5 == 0)

False